In [ ]:
import numpy as np
from rezonator import *
import matplotlib.pyplot as plt

In [ ]:
# Example resonator (see $PROJECT/bin/examples/spherical_complex.rez)

# M1  d1  GA  d2  M2
# (-------()-------)

d1 = 0.1
d2 = 0.1
F = 0.25
wl = 980e-9
a2 = 1e5
R1 = 0.1
R2 = 0.1

M_M1 = make_abcd(1, 0, -2.0/R1, 1)
M_d1 = make_abcd(1, d1, 0, 1)
M_GA = make_abcd(1, 0, -1/F - 1j*wl*a2/2.0/pi, 1)
M_d2 = make_abcd(1, d2, 0, 1)
M_M2 = make_abcd(1, 0, -2.0/R2, 1)
M0 = M_M2 * M_d2 * M_GA * M_d1 * M_M1 * M_d1 * M_GA * M_d2
print("M1\n", M_M1)
print("d1\n", M_d1)
print("GA\n", M_GA)
print("d2\n", M_d2)
print("M2\n", M_M2)
print("M0\n", M0)

In [ ]:
def stab(d1):
    M_d1 = make_abcd(1, d1, 0, 1)
    M0 = M_M2 * M_d2 * M_GA * M_d1 * M_M1 * M_d1 * M_GA * M_d2
    A, B, _, D = get_abcd(M0)
    
    gn = (A + D) / 2.0
    g = 1.0 - (A + D)**2 / 4.0
    
    q = (D - A)/2.0/B + 1j * (1/B) * np.sqrt(g)
    
    return g, gn, q

In [ ]:
d1s = np.linspace(start=0, stop=0.12, num=50)
gns_re = []
gns_im = []
gs_re = []
gs_im = []
qs = []
for d1 in d1s:
    g, gn, q = stab(d1)
    gns_re.append(gw.real)
    gns_im.append(gw.imag*1000) # bring to the same scale as real part
    gs_re.append(g.real)
    gs_im.append(g.imag*1000) # bring to the same scale as real part
    qs.append(q.imag/4) # clamp to the same scale as other graphs

# Imaginary part of self-consistent 1/q becomes zero when system gets unstable
# Both real and imaginary parts of stability parameter show specific trend at the stability boundary
# And real part of stability parameter satisfies the same conditions as in systems with pure-real matrices
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,4))

ax1.grid(True, linestyle=':')
ax1.set_xlabel('d1 (m)')
ax1.set_ylabel('1-(A+D)²/4')
ax1.plot(d1s, gs_re, label='Re')
ax1.plot(d1s, gs_im, label='Im')
ax1.plot(d1s, qs, label='Im(1/q)')
ax1.plot([stab_lim, stab_lim], [-2, 2], color='gray', linewidth=1)
ax1.legend()

ax2.grid(True, linestyle=':')
ax2.set_xlabel('d1 (m)')
ax2.set_ylabel('(A+D)/2')
ax2.plot(d1s, gws_re, label='Re')
ax2.plot(d1s, gws_im, label='Im')
ax2.plot(d1s, qs, label='Im(1/q)')
ax2.plot([0.1, 0.1], [-1.5, 1.5], color='gray', linewidth=1)
ax2.plot([0, 0.12], [-1, -1], color='gray', linewidth=1)
ax2.plot([0, 0.12], [1, 1], color='gray', linewidth=1)
ax2.legend()
